# メモ書き
## key(項目)
### jpcrp_cor:CompanyNameCoverPage
企業名
### jppfs_cor:Assets
総資産
### jpcrp_cor:NetSalesSummaryOfBusinessResults
売上高:製品や商品など有形物を販売する場合
### jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults
営業収益:無形のサービスや手数料などを提供した場合
### 例外(jpcrp030000-asr_E\*\*\*\*\*-***:NAME)
定義済みではない個社毎に独自に開示しているデータ，E\*\*\*\*\* は会社番号でNAMEに項目名が入る．  
asrはPublicDoc内の.xbrlファイル名に含まれる謎の文字列

## context_ref(詳細)
### FilingDateInstant
企業名のコンテキスト(詳細不明)
### CurrentYearInstant_NonConsolidatedMember
CurrentYearInstant:今年度  
NonConsolidatedMember:単体情報(親会社-持ち株会社のみ) ，ない場合は連結情報（子会社含む，ホールディングスと名前につくことが多い）  
Instantは総資産のみに使われていると思われる
#### CurrentYearDuration_NonConsolidatedMember
Duration:期間 は 代n期を示す  
Current:今期, Prior1:前期, Prior2:前々期, 以降さかのぼるごとにPriorに続く数値が増える

In [1]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [241]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
from bs4 import BeautifulSoup
import fnmatch
import csv
import os
import re

In [242]:
# 全xbrlファイルパスを取得する(ディレクトリを含んでしまう)
def find_all_xbrl(directory):
    for root, dirs, files in os.walk(directory):
        yield root
        for file in files:
            if fnmatch.fnmatch(file, '*.xbrl'):
                yield os.path.join(root, file)

In [243]:
# 全ファイルパスを取得
all_file_path = []
for f in find_all_xbrl("..\extracted_xbrl"):
    all_file_path.append(f)

In [244]:
# xbrl形式の入るパスのみ取得
xbrl_file_path = []
for i in all_file_path:
    # PublicDoc内のxbrlファイルはasrという文字列を含む
    if fnmatch.fnmatch(i, '*asr*.xbrl'):
        xbrl_file_path.append(i)
print("get " + str(len(xbrl_file_path)) + " file paths")

get 533 file paths


In [245]:
# パーサの初期化
parser = EdinetXbrlParser()

In [527]:
# xbrlファイルを取得してpythonで扱えるオブジェクトに格納
# 8レンゴー 18日本新薬 19日本コークス 20ツムラ 24カネカ 240住友理工 246東レ 322小林製薬 328東洋ゴム
edinet_xbrl_obj = parser.parse_file(xbrl_file_path[328]) 

In [528]:
# 企業名を取得
print(xbrl_file_path[246])
key = "jpcrp_cor:CompanyNameCoverPage"
context_ref = "FilingDateInstant"
company_name = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Company: " + company_name)

..\extracted_xbrl\Xbrl_Search_20181129_114356\S100D9YD\XBRL\PublicDoc\jpcrp030000-asr-001_E00873-000_2018-03-31_01_2018-06-26.xbrl
Company: 東洋ゴム工業株式会社


In [234]:
# 今年度の有形固定資産を取得
print(xbrl_file_path[246])
key = "jpcrp_cor:PropertyPlantAndEquipmentInformationForEachRegionTextBlock"
context_ref = "CurrentYearDuration"
tangivleFixedAssets = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("tangivleFixedAssets: " + tangivleFixedAssets)
type(tangivleFixedAssets)

..\extracted_xbrl\Xbrl_Search_20181129_114356\S100D9YD\XBRL\PublicDoc\jpcrp030000-asr-001_E00873-000_2018-03-31_01_2018-06-26.xbrl
tangivleFixedAssets: 
<p class="smt_text2" style="text-align:justify;padding-left:9pt;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;text-indent:9pt;font-family:&apos;ＭＳ 明朝&apos;;letter-spacing:0pt;line-height:15pt;"><br/>(2) 有形固定資産</p>
<div class="tbld" style="margin-left:0pt;margin-right:0pt;margin-top:0pt;margin-bottom:0pt;">
<table cellspacing="0" cellpadding="0" style="border-collapse:collapse;border:solid 0pt #000000;width:468pt;">
<colgroup>
<col style="width:78pt;min-width:78pt;"/>
<col style="width:78pt;min-width:78pt;"/>
<col style="width:78pt;min-width:78pt;"/>
<col style="width:78pt;min-width:78pt;"/>
<col style="width:78pt;min-width:78pt;"/>
<col style="width:78pt;min-width:78pt;"/>
</colgroup>
<tr style="height:9.8pt">
<td valign="bottom" style="vertical-align:bottom;border-bottom-style:solid;border-bottom-width:0.75pt;">
<p class="smt_tbl

bs4.element.NavigableString

In [47]:
# 邪魔な<br/>タグを除去
tangivleFixedAssets = str(kaigai).replace("<br/>", "")

# 有形固定資産の該当箇所の抽出
soup = BeautifulSoup(tangivleFixedAssets)
fixedAssetsList = soup.find_all("p", class_="smt_tblC")
for i in fixedAssetsList:
    print(i.string)

日本(百万円)
アジア
欧米他
合計(百万円)
韓国(百万円)
その他(百万円)
米国(百万円)
欧州他(百万円)


C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [56]:
# 総資産を取得
key = "jppfs_cor:Assets"
context_ref = "CurrentYearInstant_NonConsolidatedMember"
current_year_assets = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Assets: " + current_year_assets + " JPN")

Assets: 12768925000 JPN


In [58]:
# 売上高を取得
key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
context_ref = "CurrentYearDuration_NonConsolidatedMember"
current_year_duration = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Sales: " + current_year_duration + " JPN")

Sales: 6312682000 JPN


In [248]:
# 本社住所を取得
address_key = "jpcrp_cor:AddressOfRegisteredHeadquarterCoverPage"
address_context_ref = "FilingDateInstant"
address = edinet_xbrl_obj.get_data_by_context_ref(address_key, address_context_ref).get_value()
print("Address: " + address)

Address: 大阪市北区中之島二丁目３番18号


In [235]:
# URLテスト用
test = "わが社のホームページはhttp://www.toray.co.jp/hogeである"
res = re.findall("https?://[\w:%#\$&\?\(\)~\.=\+\-]+/", test)
print(res)

['http://www.asahi-kasei.co.jp/']


In [341]:
# 企業のホームページURLを取得
url_list = []
url_key = "jpcrp_cor:ExplanationAboutCorporateGovernanceTextBlock"
url_context_ref = "FilingDateInstant"

corporate_governance = edinet_xbrl_obj.get_data_by_context_ref(url_key, url_context_ref).get_value()
soup = BeautifulSoup(str(corporate_governance).replace("<br/>", ""))
url_tags = str(soup.find_all("p", class_="smt_text4"))

url = re.findall("https?://[\w:%#\$&\?\(\)~\.=\+\-]+/", url_tags)
# for i in range(len(url)):
#     if url[i] is not None:
#         url_list.append(url[i])

print(url)

[]


C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [529]:
tfa_key = "jpcrp_cor:PropertyPlantAndEquipmentInformationForEachRegionTextBlock"
tfa_context_ref = "CurrentYearDuration"
tangivleFixedAssets = edinet_xbrl_obj.get_data_by_context_ref(tfa_key, tfa_context_ref)

if (tangivleFixedAssets is None):
    print("pass")
    pass
else:
    tangivleFixedAssets = tangivleFixedAssets.get_value()

    # 邪魔な<br/>タグを除去して，有形固定資産の該当箇所の抽出
    soup = BeautifulSoup(str(tangivleFixedAssets).replace("<br/>", ""))
    fixedAssetsList = soup.find_all("p", class_="smt_tblC")
    for i in fixedAssetsList:
        if fnmatch.fnmatch(i.string, '*米国*') or fnmatch.fnmatch(i.string, '*北米*'):
            print("あり")
        else:
            print("なし")
print(fixedAssetsList)

なし
なし
なし
なし
なし
なし
あり
なし
なし
あり
なし
なし
なし
[<p class="smt_tblC"> </p>, <p class="smt_tblC"> </p>, <p class="smt_tblC"> </p>, <p class="smt_tblC"> </p>, <p class="smt_tblC"> </p>, <p class="smt_tblC">日本</p>, <p class="smt_tblC">北米</p>, <p class="smt_tblC">その他</p>, <p class="smt_tblC">合計</p>, <p class="smt_tblC">米国</p>, <p class="smt_tblC">その他</p>, <p class="smt_tblC">マレーシア</p>, <p class="smt_tblC">その他</p>]


C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [534]:
# 売上が1000億円以上の企業の各データを取得
target_company = [] # 会社名
target_sales = []   # 売上高
target_address = [] # 本社住所
target_url = []     # URL
target_assets = []  # 有形固定資産

company_context_ref = "FilingDateInstant"
sales_context_ref = "CurrentYearDuration_NonConsolidatedMember" # 単体情報(親会社のみ)
address_context_ref = "FilingDateInstant"
url_context_ref = "FilingDateInstant"
tfa_context_ref = "CurrentYearDuration"

company_key = "jpcrp_cor:CompanyNameCoverPage"
address_key = "jpcrp_cor:AddressOfRegisteredHeadquarterCoverPage"
url_key = "jpcrp_cor:OverviewOfOperationalProceduresForSharesTextBlock"
# other_url_key = "jpcrp_cor:ExplanationAboutCorporateGovernanceTextBlock"
tfa_key = "jpcrp_cor:PropertyPlantAndEquipmentInformationForEachRegionTextBlock"

for xi in xbrl_file_path:
    sales_key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
    
    # 指定オブジェクトの取得
    edinet_xbrl_obj = parser.parse_file(xi)
    sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高が存在しないなら，営業収益を取得
    if sales is None:
        sales_key = "jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults"
        sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高，営業収益ともにないなら除外
    if sales is None:
        pass
    else:
        sales = sales.get_value()
    
    if sales is None:
        pass
    else:
        # 1000億円以上の売上ならリストに追加
        if int(sales) >= 100000000000:
            tags = 0
            assets = 0
            assets_flag = False
            
            company = edinet_xbrl_obj.get_data_by_context_ref(company_key, company_context_ref).get_value()
            address = edinet_xbrl_obj.get_data_by_context_ref(address_key, address_context_ref).get_value()
            tangivleFixedAssets = edinet_xbrl_obj.get_data_by_context_ref(tfa_key, tfa_context_ref)
            
            target_address.append(address)
            
            # URL取得
            corporate_governance = edinet_xbrl_obj.get_data_by_context_ref(url_key, url_context_ref).get_value()
            soup = BeautifulSoup(str(corporate_governance).replace("<br/>", ""))
            url_tags = soup.find_all("p") # 一つのxbrlファイルの該当箇所のpタグ全てのリスト (参考)class_="smt_text4" 東レ

            for ui in url_tags:
                options = str(ui.string)
                url = re.findall("https?://[\w:%#\$&\?\(\)~\.=\+\-]+/", options)
                if url != []:
                    target_url.append(url[0])
                    continue
                elif tags == (len(url_tags) - 1):
                    target_url.append("")
                tags += 1
            
            if (tangivleFixedAssets is None):
                target_assets.append("")
            else:
                tangivleFixedAssets = tangivleFixedAssets.get_value()
                
                # 邪魔な<br/>タグを除去して，有形固定資産の該当箇所の抽出
                soup = BeautifulSoup(str(tangivleFixedAssets).replace("<br/>", ""))
                fixedAssetsList = soup.find_all("p", class_="smt_tblC")
                
                if fixedAssetsList == []:
                    target_assets.append("")
                else:
                    for ai in fixedAssetsList:
                        if assets_flag==False and (fnmatch.fnmatch(ai.string, '*米国*') or fnmatch.fnmatch(ai.string, '*北米*')):
                            target_assets.append("あり")
                            assets_flag = True
                            continue
                        elif assets == (len(fixedAssetsList) - 1):
                            target_assets.append("")
                        assets += 1
            
            target_company.append(company)
            target_sales.append(sales)

print(target_company)
print(target_sales)
print(target_address)
print(target_url)
print(target_assets)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


株式会社ツムラ
宇部興産株式会社
出光興産株式会社
信越化学工業株式会社
関西ペイント株式会社
大王製紙株式会社
出光興産株式会社
積水化学工業株式会社
日油株式会社
富士石油株式会社
日本ゼオン株式会社
大日精化工業株式会社
日本ゼオン株式会社
アイカ工業株式会社
株式会社ＡＤＥＫＡ
三洋化成工業株式会社
株式会社トクヤマ
株式会社ダイセル
三菱瓦斯化学株式会社
三井化学株式会社
小林製薬株式会社
ＤＩＣ株式会社
昭和電工株式会社
小林製薬株式会社
トヨタ紡織株式会社
豊田合成株式会社
帝人株式会社
関西ペイント株式会社
大王製紙株式会社
信越化学工業株式会社
株式会社ブリヂストン
['レンゴー株式会社', '日本新薬株式会社', '日本コークス工業株式会社', '株式会社ツムラ', '株式会社カネカ', '宇部興産株式会社', '武田薬品工業株式会社', '日本製紙株式会社', '出光興産株式会社', '信越化学工業株式会社', '関西ペイント株式会社', '大王製紙株式会社', '日立化成株式会社', '出光興産株式会社', '日本コークス工業株式会社', '旭化成株式会社', 'エア・ウォーター株式会社', '積水化学工業株式会社', '日油株式会社', '株式会社エフピコ', '三菱製紙株式会社', '日本化薬株式会社', '北越紀州製紙株式会社', '日産化学工業株式会社', '富士石油株式会社', '東ソー株式会社', '日本ゼオン株式会社', '大日精化工業株式会社', '持田製薬株式会社', '株式会社コーセー', '日本ゼオン株式会社', 'アイカ工業株式会社', '株式会社ＡＤＥＫＡ', '日東電工株式会社', '住友理工株式会社', '東レ株式会社', '三洋化成工業株式会社', 'ダイキョーニシカワ株式会社', '田辺三菱製薬株式会社', '小野薬品工業株式会社', '株式会社トクヤマ', '株式会社ダイセル', 'ハリマ化成グループ株式会社', '三菱瓦斯化学株式会社', '東洋紡株式会社', 'グンゼ株式会社', '参天製薬株式会社', '沢井製薬株式会社', '三井化学株式会社', '小林製薬株式会社', '東洋ゴム工業株式会社', '横浜ゴム株式会社', 'ＤＩＣ株式会社', '昭和電工株式会社', 'ライオン株式会社', '住友ゴ

In [535]:
print(len(target_company))
print(len(target_sales))
print(len(target_address))
print(len(target_url))
print(len(target_assets))

91
91
91
91
91


In [537]:
# csv出力用リストの作成
list4csv = [['企業名', '売上高(JPY)', '本店住所', '企業URL', '米国拠点']]
for i in range(len(target_company)):
    list4csv.append([target_company[i], target_sales[i], target_address[i].replace("\n", ""), target_url[i], target_assets[i]])
print(list4csv)

[['企業名', '売上高(JPY)', '本店住所', '企業URL', '米国拠点'], ['レンゴー株式会社', '303406000000', '大阪市福島区大開四丁目１番186号(上記の住所は登記上のものであり、実際の業務は下記の場所で行っている。)大阪市北区中之島二丁目２番７号\u3000中之島セントラルタワー', 'http://www.rengo.co.jp/', ''], ['日本新薬株式会社', '101221000000', '京都市南区吉祥院西ノ庄門口町14番地', 'http://www.nippon-shinyaku.co.jp/', ''], ['日本コークス工業株式会社', '100472000000', '東京都江東区豊洲三丁目３番３号', '', ''], ['株式会社ツムラ', '116230000000', '東京都港区赤坂二丁目17番11号', '', ''], ['株式会社カネカ', '293016000000', '大阪市北区中之島二丁目３番18号', 'http://www.kaneka.co.jp/', 'あり'], ['宇部興産株式会社', '315241000000', '山口県宇部市大字小串1978番地の96', '', ''], ['武田薬品工業株式会社', '659462000000', '大阪府大阪市中央区道修町四丁目１番１号（上記は登記上の本店所在地で実際の業務は「最寄りの連絡場所」で行っております。）', 'https://www.takeda.com/', ''], ['日本製紙株式会社', '611735000000', '東京都北区王子一丁目４番１号(上記は登記上の本店所在地であり実際の本社業務は下記にて行っています。)東京都千代田区神田駿河台四丁目６番地', 'http://www.nipponpapergroup.com/', 'あり'], ['出光興産株式会社', '2746981000000', '東京都千代田区丸の内三丁目１番１号', '', ''], ['信越化学工業株式会社', '733449000000', '東京都千代田区大手町二丁目６番１号', 'http://www.shinetsu.co.jp/', ''], ['関西ペイント株式会社', '151983000000', 

In [538]:
with open('company_list.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(list4csv)